In [1]:
import numpy as np
import dicom
import sys
sys.path.append('..')
import os
from AlgoEngine.utils import getContours, getImageBlock, convertROIToCTSpace
from General.testing_utils import getContourInputs
from AlgoEngine.ovh import getHistogram, getOVHDistances
from math import sqrt

## Inputs to Function

In [2]:
BASE_DIR = '/home/radiation/RadiationTherapyDecisionSupport/data/'
StudyID = 'UCLA_PR_6'
_, sop_ids = getImageBlock(StudyID, BASE_DIR)

ctFilenames = [fl for fl in os.listdir(BASE_DIR + StudyID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + StudyID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows
row_spacing = float(sampleCTImage.PixelSpacing[0])
column_spacing = float(sampleCTImage.PixelSpacing[1])
slice_spacing = float(sampleCTImage.SliceThickness)

In [3]:
ROI_NAME = 'PTV'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(BASE_DIR, StudyID, ROI_NAME, excluding=[])
ptv_contour_block, ptv_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
ptv_contour_block = convertROIToCTSpace(ptv_contour_block, image_position, sop_ids)
ptv_roi_block = convertROIToCTSpace(ptv_roi_block, image_position, sop_ids)

ROI_NAME = 'Bladder'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(BASE_DIR, StudyID, ROI_NAME, excluding=[])
_, oar_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
oar_roi_block = convertROIToCTSpace(oar_roi_block, image_position, sop_ids)

In [4]:
oar_dists = getOVHDistances(oar_roi_block, ptv_contour_block, ptv_roi_block, row_spacing, column_spacing, slice_spacing)

In [5]:
n_bins = 10 # Input parameter

## Function Starts Here

In [6]:
bin_vals, bin_amts = getHistogram(oar_dists, oar_roi_block, n_bins)

## Testing Function

We assert that the number of pixels in each bin total equals the number of pixels in the OAR ROI in total.

In [7]:
sum_bins = np.sum(bin_amts)
print(np.allclose(sum_bins, np.count_nonzero(oar_roi_block)))

True
